In [70]:
import vedo
import os
import numpy as np
import napari
from skimage import filters, io

from napari_stress import reconstruction, measurements
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

In [2]:
viewer = napari.Viewer()

Make folders:

In [33]:
os.makedirs('./results_sphere', exist_ok=True)
os.makedirs('./results_ellipsoid', exist_ok=True)

for path in ['./results_sphere', './results_ellipsoid']:
    os.makedirs(os.path.join(path, 'napari_stress_analysis_and_napari_stress_pointcloud'), exist_ok=True)
    os.makedirs(os.path.join(path, 'napari_stress_analysis_and_stress_pointcloud'), exist_ok=True)
    os.makedirs(os.path.join(path, 'stress_analysis_and_stress_pointcloud'), exist_ok=True)

## Create synthetic data

Create sphere with padded border

In [3]:
radius = 5
padding = 15
sigma = 5

In [10]:
sphere = vedo.Sphere(r=radius).binarize(spacing=(0.25, 0.25, 0.25)).tonumpy()
sphere = np.pad(sphere, padding)
sphere = filters.gaussian(sphere, sigma=sigma)  # blur

In [11]:
io.imsave(os.path.join('.', 'results_sphere', f'synthetic_sphere_radius_{radius}_padding_{padding}_sigma_{sigma}.tif'), sphere)

Repeat for ellipsoid:

In [24]:
radius_major = 10
radius_minor = 7.5
ellipsoid = vedo.Ellipsoid(
    axis1=(radius_major, 0,0), axis2=(0, radius_minor, 0), axis3=(0, 0, radius_minor)
    ).binarize(spacing=(0.25, 0.25, 0.25)).tonumpy()

ellipsoid = np.pad(ellipsoid, padding)
ellipsoid = filters.gaussian(ellipsoid, sigma=sigma)  # blur

In [25]:
io.imsave(os.path.join('.', 'results_ellipsoid', f'synthetic_ellipsoid_radius_major_{radius_major}_radius_minor_{radius_minor}_padding_{padding}_sigma_{sigma}.tif'), ellipsoid)

In [26]:
viewer.add_image(sphere)

<Image layer 'sphere' at 0x20b0e750040>

## Analyze with napari-stress

### Sphere

In [19]:
dest = os.path.join('./results_sphere/napari_stress_analysis_and_napari_stress_pointcloud/')

Sphere:

In [ ]:
results_reconstruction = reconstruction.reconstruct_droplet(sphere,
                                                            voxelsize=np.asarray([1, 1, 1]),
                                                            target_voxelsize=1,
                                                            n_smoothing_iterations=15,
                                                            n_points=256,
                                                            n_tracing_iterations=3,
                                                            resampling_length=1,
                                                            fit_type='fancy',
                                                            edge_type='interior',
                                                            trace_length=20,
                                                            sampling_distance=1,
                                                            remove_outliers=False,
                                                            verbose=False,
                                                            use_dask=False
                                                            )

refined_points = results_reconstruction[3][0]
results_stress_analysis_sphere = measurements.comprehensive_analysis(refined_points,
                                                              max_degree=20,
                                                              n_quadrature_points=590,
                                                              gamma=5,
                                                              verbose=False,
                                                              use_dask=False)

for res in results_reconstruction + results_stress_analysis_sphere:
    if res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    if res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])


In [20]:
np.savetxt(os.path.join(dest, 'mean_curvatures.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['mean_curvature'])
np.savetxt(os.path.join(dest, 'total_stress.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['anisotropic_stress'])
np.savetxt(os.path.join(dest, 'lebedev_points.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].data)
np.savetxt(os.path.join(dest, 'expanded_points.csv'), viewer.layers['Result of fit spherical harmonics (deg = 20'].data)
np.savetxt(os.path.join(dest, 'pointcloud.csv'), refined_points)

In [29]:
viewer.close_all()
viewer = napari.Viewer(ndisplay=3)

### Ellipsoid:

In [27]:
dest = os.path.join('./results_ellipsoid/napari_stress_analysis_and_napari_stress_pointcloud/')

In [28]:
results_reconstruction = reconstruction.reconstruct_droplet(ellipsoid,
                                                            voxelsize=np.asarray([1, 1, 1]),
                                                            target_voxelsize=1,
                                                            n_smoothing_iterations=15,
                                                            n_points=256,
                                                            n_tracing_iterations=3,
                                                            resampling_length=1,
                                                            fit_type='fancy',
                                                            edge_type='interior',
                                                            trace_length=20,
                                                            sampling_distance=1,
                                                            remove_outliers=False,
                                                            verbose=False,
                                                            use_dask=False
                                                            )

refined_points = results_reconstruction[3][0]
results_stress_analysis_ellipsoid = measurements.comprehensive_analysis(refined_points,
                                                              max_degree=20,
                                                              n_quadrature_points=590,
                                                              gamma=5,
                                                              verbose=False,
                                                              use_dask=False)


for res in results_reconstruction + results_stress_analysis_ellipsoid:
    if res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    if res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])


In [31]:
np.savetxt(os.path.join(dest, 'mean_curvatures.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['mean_curvature'])
np.savetxt(os.path.join(dest, 'total_stress.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].features['anisotropic_stress'])
np.savetxt(os.path.join(dest, 'lebedev_points.csv'), viewer.layers['Result of lebedev quadrature (droplet)'].data)
np.savetxt(os.path.join(dest, 'expanded_points.csv'), viewer.layers['Result of fit spherical harmonics (deg = 20'].data)
np.savetxt(os.path.join(dest, 'pointcloud.csv'), refined_points)